In [20]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import re

import warnings


In [27]:
# DO NOT CHANGE
def load_and_check_dataset(file_path, building_name, year, check_date = False):
    df = pd.read_excel(file_path)
    
    expected_columns = ["name", "slottime_GMT", "pointTitle", "value"]
    expected_building_name = building_name
    
    # Raw dataset check
    if df.shape[1] != 4:
        raise ValueError(f"Dataset must contain exactly 4 columns, but it has {df.shape[1]} columns. It might not be a raw dataset")
    if list(df.columns) != expected_columns:
        raise ValueError(f"Dataset columns are not in the expected order. Expected {expected_columns}, but got {list(df.columns)}.")
        
    # Building Check
    if not df['name'].str.startswith(expected_building_name).all():
        raise ValueError("The expected building name does not match to the building names in the dataset.")
        
        #warnings.warn("the expected building name does not match to the building names in the dataset.")
        #override = input("Do you want to override this warning and proceed? (yes/no): ")
        #if override.lower() != 'yes':
            #print("Override not confirmed. Exiting process.")
            #raise SystemExit
        #else:
            #print("Override confirmed. Proceeding with the process.")
    
    # Missing Data Check
    if len(df) >= 9999:
        raise ValueError("DataFrame length is 9999 or more. This might be due to excessive data downloads. Try download the data following the instructions.")
        
        #warnings.warn("DataFrame length is 9999 or more. This might due to excessive data downloads.")
        #override = input("Do you want to override this warning and proceed? (yes/no): ")
        #if override.lower() != 'yes':
            #print("Override not confirmed. Exiting process.")
            #raise SystemExit
        #else:
            #print("Override confirmed. Proceeding with the process.")
    
    if check_date:
        datetime_output = pd.to_datetime(df['slottime_GMT'])        
        last_date_minus_one = datetime_output.iloc[-1] - pd.Timedelta(days=1)
        formatted_date = last_date_minus_one.strftime('%m-%d-%Y')
        formatted_year = last_date_minus_one.strftime('%Y')

        if (datetime_output[0] != file_path[-29:-19]) or (formatted_date != file_path[-15:-5]):
            raise ValueError("The date range does not match to the file's name.")
        
        if (formatted_year != str(year)):
            raise ValueError("The year specified does not match to the file's data.")
    
    return None

In [41]:
# DO NOT CHANGE
def pivot_merge_data(building_name, year):
    
    # MAKE SURE YOUR BUILDING NAME MATCHES TO THE PORTAL
    folder_path = 'DataFiles_' + building_name + '_' + str(year)
    
    dfs = []
    
    for filename in os.listdir(folder_path):
        
        # Find all Excel Files
        if filename.endswith('.xlsx') and (not filename.startswith("~$")):
            file_path = os.path.join(folder_path, filename)
            
            # Load and check dataset
            load_and_check_dataset(file_path, building_name, year)
            
            dfs.append(pd.read_excel(file_path))
    
    merged_df = pd.concat(dfs, ignore_index=True)
    
    merged_df = merged_df.drop_duplicates()
    
    merged_df['name'] = building_name
    merged_df.rename(columns={'slottime_GMT': 'slottime'}, inplace=True)
    pivot_df = merged_df.pivot_table(index = ['name', 'slottime'], columns='pointTitle', values='value').reset_index()
    pivot_df = pivot_df.drop('Clean Room Dehumidifier Steam', axis = 1)
    
    pivot_df['slottime'] = pd.to_datetime(pivot_df['slottime'])
    pivot_df = pivot_df.sort_values(by = 'slottime')

    # Check dates
    datetime_slottime = pivot_df['slottime'].dt.strftime('%m-%d-%Y')
    datetime_slottime = pd.to_datetime(datetime_slottime, format='%m-%d-%Y')

    year_to_check = year

    start_date = f"{year_to_check}-01-01"
    end_date = f"{year_to_check}-12-31"
    full_date_range = pd.date_range(start=start_date, end=end_date)

    missing_dates = full_date_range[~full_date_range.isin(datetime_slottime)]

    if not missing_dates.empty:
        warning_message = f"Warning: There are {len(missing_dates)} dates missing for the year {year_to_check}. "
        warning_message += "First five missing dates: "
        warning_message += ', '.join(missing_dates[:5].strftime('%m-%d-%Y'))
        warnings.warn(warning_message)
        override = input("Do you want to override this warning and proceed? (yes/no): ")
        if override.lower() != 'yes':
            print("Override not confirmed. Exiting process.")
            raise SystemExit
        else:
            print("Override confirmed. Proceeding with the process.")
    
    return pivot_df

In [23]:
# DO NOT CHANGE
def save_df_to_csv(building_name, year):
    file_path = "CleanedDataFiles/" + building_name + "_" + str(year) + '.xlsx'
    df = pivot_merge_data(building_name, year)
    df.to_excel(file_path, index=False)

Modify the code below to pivot and merge code!

In [42]:
'''
building_name_input = "DuffieldHall" # Replace this name with the EXACT building name shown on the portal
year_input = 2023 # Replace this year with the collected year

save_df_to_csv(building_name_input, year_input)
'''

/var/folders/gm/h4qjw7x550z6d6z8qtv4tyj40000gn/T/ipykernel_2765/924606708.py:48: UserWarning: Warning: There are 303 dates missing for the year 2023. First five missing dates: 02-02-2023, 02-03-2023, 02-04-2023, 02-05-2023, 02-06-2023
  warnings.warn(warning_message)


Do you want to override this warning and proceed? (yes/no): yes
Override confirmed. Proceeding with the process.
